In [1]:
import pandas as pd
from glob import glob
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim
import os
import math
import shutil

/Users/hyunsucksong/opt/anaconda3/envs/songforgpu/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = os.path.join(os.getcwd(), '계측_csv_final')
data_path_list = glob(os.path.join(data_path, '*.csv'))
print(len(data_path_list))

0


In [4]:
using_columns = ['Time[s]', 'TE1', 'E.RPM',
       'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar','E.Id','RPM', 'P_UD']

x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar']
y_columns = ['P_UD']

In [5]:
data_size = len(data_path_list)
train_size = int(data_size*0.8)
test_size = data_size - train_size
val_size = int(train_size*0.2)

In [6]:
train_datapath, test_datapath = random_split(data_path_list, [train_size, test_size])
train_datapath, val_datapath = random_split(train_datapath, [(train_size - val_size),val_size])

In [2]:
def createFolder(directory): #현재 경로에 Train / Test / Val 폴더 생성
    if not os.path.exists(directory):
        os.mkdir(directory)
        
    elif os.path.exists(directory):
        print('Already exists')
        
    else:
        print('Error: Creating directory. ' + directory)
        
createFolder(os.path.join(os.getcwd(),'csv_final/train'))
createFolder(os.path.join(os.getcwd(),'csv_final/test'))
createFolder(os.path.join(os.getcwd(),'csv_final/val'))

Already exists
Already exists
Already exists


In [ ]:
for i in range(len(train_datapath)):
    shutil.copy2(train_datapath[i], './csv_final/train')
    
for j in range(len(test_datapath)):
    shutil.copy2(test_datapath[j], './csv_final/test')
    
for k in range(len(val_datapath)):
    shutil.copy2(val_datapath[k], './csv_final/val')

In [49]:
train=[]
test=[]
val=[]

for path in train_datapath :
    df_cut = pd.read_csv(path)[using_columns]
    train.append(df_cut)
    
for path in test_datapath :
    df_cut = p2d.read_csv(path)[using_columns]
    test.append(df_cut)
    
for path in val_datapath :
    df_cut = pd.read_csv(path)[using_columns]
    val.append(df_cut)

In [62]:
def window_torch(df, x_size, shift): # size = 윈도우 사이즈 / shift = 윈도우 시작점 얼만큼 이동할건지
    df_X =False
    for i in range(len(df)):
        if df_X is False:
            df_X = torch.tensor(df[i][x_columns].values, dtype=torch.float32)
            df_X = df_X.unfold(0, x_size, shift).transpose(1,2)
            
        else:
            tmp1 = torch.tensor(df[i][x_columns].values, dtype=torch.float32)
            tmp1 = tmp1.unfold(0,x_size, shift).transpose(1,2)
            df_X = torch.cat([df_X, tmp1],0)
            
    df_Y = False
    for i in range(len(df)):
        if df_Y is False:
            df_Y = torch.tensor(df[i][y_columns][(x_size-1):].values, dtype=torch.float32)
            df_Y = df_Y.unfold(0, 1, shift).transpose(1,2)
        
        else:
            tmp2 = torch.tensor(df[i][y_columns][(x_size-1):].values, dtype=torch.float32)
            tmp2 = tmp2.unfold(0, 1, shift).transpose(1,2)
            df_Y = torch.cat([df_Y, tmp2],0)
            
    return df_X, df_Y

In [63]:
x_size, shift = 2, 1

In [64]:
train_X, train_Y = window_torch(train, x_size, shift)
train_dataset = torch.utils.data.TensorDataset(train_X, train_Y)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1, shuffle=False) 

test_X, test_Y = window_torch(test, x_size, shift)
test_dataset = torch.utils.data.TensorDataset(test_X, test_Y)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False) 

val_X, val_Y = window_torch(val, x_size, shift)
val_dataset = torch.utils.data.TensorDataset(val_X, val_Y)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

In [71]:
for i, data in enumerate(test_loader):
    print(data)
    break

[tensor([[[  28.2280,  994.0000,   13.0000,    2.9600,   73.0000,    5.0000,
          2922.0000],
         [  28.2280,  994.0000,   13.0000,    2.9600,   73.0000,    5.0000,
          2922.0000]]]), tensor([[[1.5010]]])]


In [72]:
print(len(list(train_X)))
print(len(list(train_Y)))

1558538
1558538
